In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from karateclub import Graph2Vec
from sklearn.linear_model import SGDClassifier
import pandas as pd
from sklearn.linear_model import SGDClassifier

np.set_printoptions(precision=3)
pd.set_option('display.max_colwidth', None)
std = StandardScaler()


In [4]:
with open('data/REDDIT-BINARY_graph2vec_classification_train.npy', 'rb') as f:
    redit_train = np.load(f, allow_pickle=True)

with open('data/REDDIT-BINARY_graph2vec_classification_test.npy', 'rb') as f:
    reddit_test = np.load(f, allow_pickle=True)

with open('data/PROTEINS_graph2vec_classification_test.npy', 'rb') as f:
    unknown_test = np.load(f, allow_pickle=True)

with open('data/PROTEINS_graph2vec_classification_train.npy', 'rb') as f:
    unknown_train = np.load(f, allow_pickle=True)


with open('data/IMDB-BINARY_graph2vec_classification_test.npy', 'rb') as f:
    unknown_test_imdb = np.load(f, allow_pickle=True)

with open('data/IMDB-BINARY_graph2vec_classification_train.npy', 'rb') as f:
    unknown_train_imdb = np.load(f, allow_pickle=True)


# SGDclassifier
- When inclue multiclass, automaticaly use OvA
- Able to get score percentage by using min-max normalisation

# Reference
- https://www.youtube.com/watch?v=5KyH6v8oKNQ

In [5]:
def train_classif(model, epoch, x_embed, y_label, unique_lbl):
    for i in range(epoch):
        model.partial_fit(x_embed, y_label, unique_lbl)

    return model

def predict_classif(model, x_test_embed):
    y_pred = model.predict([x_test_embed]).squeeze()
    y_proba = model.decision_function([x_test_embed])
    y_proba = (y_proba - y_proba.min()) / (y_proba.max() - y_proba.min())
    y_proba = [x / np.sum(y_proba) for x in y_proba.squeeze()]
    y_proba = y_proba[y_pred] * 100
    y_proba = np.round(y_proba, 3)
    return y_pred, y_proba


In [6]:
# Train with REDDIT data as unauth access+data theft with PROTEINS(train and test so have more sample->more accurate) as unknown
unknown_protein_label = []
# Replace with synthetic label for PROTEINS to equate it to unknwon type class(2==unknwon)
for  y in [y for x, y, g in unknown_test] + [y for x, y, g in unknown_train]:
    if y == 0:
        unknown_protein_label.append(2)
    if y == 1:
        unknown_protein_label.append(2)

X_train = [x for x, y, g in redit_train] + [x for x, y,
                                            g in unknown_test] + [x for x, y, g in unknown_train]
std_model = std.fit(X_train)
X_train = std_model.transform(X_train)
y_train = [y for x, y, g in redit_train] + unknown_protein_label

X_test = [x for x, y, g in unknown_test]
X_test = std_model.transform(X_test)
y_test = unknown_protein_label


In [7]:
# Test to check if given unknwon data, it should predict only 2, never 0 or 1
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train, y_train, [0, 1, 2])
pred, score = predict_classif(model, X_test[2])
print("predicted:",pred, " with score:",score, " actual:", y_test[2])


predicted: 2  with score: 74.006  actual: 2


In [8]:
# Test to check if given knwon data, it should predict 0 or 1, never 2
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train, y_train, [0, 1, 2])
pred, score = predict_classif(model, X_train[2])
print("predicted:", pred, " with score:", score, " actual:", y_train[2])


predicted: 0  with score: 71.529  actual: 0


In [9]:
# Now train with another data IMDB(train only) to test if the data is predicted as unknown
unknown_train_label_imdb = []
for x,y,g in unknown_train_imdb:
    if y == 0:
        unknown_train_label_imdb.append(2)
    if y == 1:
        unknown_train_label_imdb.append(2)

X_train_withunknown = [x for x, y, g in unknown_train_imdb]
X_train_withunknown = std_model.transform(X_train_withunknown)
y_train_withunknown = unknown_train_label_imdb


In [10]:
# Test to check if given unknwon data, it should predict only 2, never 0 or 1
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train, y_train, [0, 1, 2])
pred, score = predict_classif(model, X_train_withunknown[0])
print("predicted:", pred, " with score:",
      score, " actual:", y_train_withunknown[0])


predicted: 2  with score: 64.424  actual: 2


In [11]:
# Finally test with known sample REDDIT(test) to check if predicts either of the known classes 0 or 1
X_train_known = [x for x, y, g in reddit_test]
X_train_known = std_model.transform(X_train_known)
y_train_known = [ y for x, y, g in reddit_test]


In [12]:
# Test to check if given knwon data, it should predict 0 or 1, never 2
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train, y_train, [0, 1, 2])
pred, score = predict_classif(model, X_train_known[4])
print("predicted:", pred, " with score:",
      score, " actual:", y_train_known[4])


predicted: 0  with score: 58.386  actual: 0
